<a href="https://colab.research.google.com/github/akshat2635/DL/blob/main/Word_Predictor_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d ronikdedhia/next-word-prediction

Dataset URL: https://www.kaggle.com/datasets/ronikdedhia/next-word-prediction
License(s): unknown
next-word-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d shiv28/next-word

Dataset URL: https://www.kaggle.com/datasets/shiv28/next-word
License(s): unknown
next-word.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/next-word-prediction.zip', 'r')
zip_ref.extractall('/content/data')
zip_ref.close()

In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/next-word.zip', 'r')
zip_ref.extractall('/content/data')
zip_ref.close()

In [6]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
filename="/content/data/next_word_prediction.txt"
story=[]
f=open(filename,'r', encoding='latin-1')
text=f.read()
sent=sent_tokenize(text)
for i in sent:
  story.append(simple_preprocess(i))

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(story)

In [9]:
len(tokenizer.word_index)

11117

In [10]:
train_data=[]
for sent in story:
  token_sent=tokenizer.texts_to_sequences([sent])[0]
  for i in range(1,len(token_sent)):
    train_seq=token_sent[:i+1]
    train_data.append(train_seq)

In [11]:
max_len = max([len(x) for x in train_data])

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_data=pad_sequences(train_data,maxlen=max_len,padding='pre')

In [13]:
X=train_data[:,:-1]
y=train_data[:,-1]

In [14]:
y.shape

(72883,)

In [15]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=len(tokenizer.word_index)+1)
y.shape

(72883, 11118)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [17]:
model=Sequential()
model.add(Embedding(len(tokenizer.word_index)+1,100,input_length=max_len-1))
model.add(LSTM(150))
# model.add(Dense(100,activation='relu'))
model.add(Dense(len(tokenizer.word_index)+1,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model.fit(X,y,epochs=20,validation_split=0.2)

Epoch 1/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 42s 19ms/step - accuracy: 0.0109 - loss: 8.5220 - val_accuracy: 0.0134 - val_loss: 8.6826
Epoch 2/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 38s 19ms/step - accuracy: 0.0221 - loss: 7.8422 - val_accuracy: 0.0234 - val_loss: 8.6487
Epoch 3/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 38s 18ms/step - accuracy: 0.0316 - loss: 7.4036 - val_accuracy: 0.0263 - val_loss: 8.6460
Epoch 4/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 42s 18ms/step - accuracy: 0.0429 - loss: 6.9228 - val_accuracy: 0.0306 - val_loss: 8.8365
Epoch 5/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 33s 18ms/step - accuracy: 0.0591 - loss: 6.3708 - val_accuracy: 0.0396 - val_loss: 9.1040
Epoch 6/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.0936 - loss: 5.7993 - val_accuracy: 0.0491 - val_loss: 9.3347
Epoch 7/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 44s 20ms/step - accuracy: 0.1527 - loss: 5.2409 - val_accuracy: 0.0589 - val_loss: 9.5304
Epoch 8/20
1823/1823 ━━━━━━━━━━━━━━━━━━━━ 38s 18ms/step - accuracy: 0.2235 -

In [19]:
import time
text = "the man there has a very long"

for i in range(20):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=max_len-1, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
the man there has a very long time
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
the man there has a very long time said
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
the man there has a very long time said lord
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
the man there has a very long time said lord henry
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
the man there has a very long time said lord henry smiling
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
the man there has a very long time said lord henry smiling looking
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
the man there has a very long time said lord henry smiling looking room
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
the man there has a very long time said lord henry smiling looking room table
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
the man there has a very long time said lord henry smiling looking room table examined
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
the man there has a very long time said lord henry smiling looking room table examine

In [21]:
import time
text = "i am"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=max_len-1, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
i am like
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
i am like that
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
i am like that he
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
i am like that he turning
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
i am like that he turning elder
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
i am like that he turning elder woman
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
i am like that he turning elder woman grew
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
i am like that he turning elder woman grew rough
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
i am like that he turning elder woman grew rough beneath
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
i am like that he turning elder woman grew rough beneath eyes
